In [ ]:
import numpy as np
import pandas as pd
from string import digits
import re
import string
import nltk
nltk.download("punkt")
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import csv

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/Kids_summary/summaries_train.csv")
df2=pd.read_csv("/content/drive/MyDrive/Kids_summary/prompts_train.csv")
df3=pd.read_csv("/content/drive/MyDrive/Kids_summary/summaries_test.csv")
df4=pd.read_csv("/content/drive/MyDrive/Kids_summary/prompts_test.csv")

In [ ]:
merge_df=pd.merge(df,df2, on='prompt_id',how='inner')

In [ ]:
merge_df=merge_df.loc[:,['student_id','prompt_id','prompt_question','prompt_text','text','content','wording']]
merge_df

,student_id,prompt_id,prompt_question,prompt_text,text,content,wording
0,000e8c3c7ddb,814d6b,Summarize how the Third Wave developed over su...,Background \r\nThe Third Wave experiment took ...,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0070c9e7af47,814d6b,Summarize how the Third Wave developed over su...,Background \r\nThe Third Wave experiment took ...,The Third Wave developed rapidly because the ...,3.272894,3.219757
2,0095993991fe,814d6b,Summarize how the Third Wave developed over su...,Background \r\nThe Third Wave experiment took ...,The third wave only started as an experiment w...,0.205683,0.380538
3,00c20c6ddd23,814d6b,Summarize how the Third Wave developed over su...,Background \r\nThe Third Wave experiment took ...,The experimen was orginally about how even whe...,0.567975,0.969062
4,00d40ad10dc9,814d6b,Summarize how the Third Wave developed over su...,Background \r\nThe Third Wave experiment took ...,The third wave developed so quickly due to the...,-0.910596,-0.081769
...,...,...,...,...,...,...,...
7160,fef3e85236e5,39c16e,Summarize at least 3 elements of an ideal trag...,Chapter 13 \r\nAs the sequel to what has alrea...,"It has to be made on a complex storyline, with...",-0.981265,-1.548900
7161,ff0f65eecf02,39c16e,Summarize at least 3 elements of an ideal trag...,Chapter 13 \r\nAs the sequel to what has alrea...,Aristotle descirbes an ideal tradgedy as being...,-0.511077,-1.589115
7162,ff186473ea0a,39c16e,Summarize at least 3 elements of an ideal trag...,Chapter 13 \r\nAs the sequel to what has alrea...,A tragedy should have a complex plan not a sim...,-0.834946,-0.593749
7163,ff5e9e6068da,39c16e,Summarize at least 3 elements of an ideal trag...,Chapter 13 \r\nAs the sequel to what has alrea...,Aristotle believed that the ideal tradegy shou...,-0.157460,-0.165811


In [ ]:
merge_df['prompt_question']=merge_df['prompt_question'].astype(str)
merge_df['prompt_question']=merge_df['prompt_question'].apply(lambda x:x.lower())
merge_df['prompt_question']=merge_df['prompt_question'].apply(lambda x:re.sub("'"," ", x))
merge_df['prompt_question']=merge_df['prompt_question'].apply(lambda x:x.strip())
merge_df['prompt_question']=merge_df['prompt_question'].apply(lambda x:re.sub(" +"," ",x))
exclude=set(string.punctuation)
merge_df['prompt_question']=merge_df['prompt_question'].apply(lambda x:''.join(ch for ch in x if ch not in exclude))
remove=str.maketrans('','',digits)
merge_df['prompt_question']=merge_df['prompt_question'].apply(lambda x:x.translate(remove))

In [ ]:
merge_df['text']=merge_df['text'].astype(str)
merge_df['text']=merge_df['text'].apply(lambda x:x.lower())
merge_df['text']=merge_df['text'].apply(lambda x:re.sub("'"," ", x))
merge_df['text']=merge_df['text'].apply(lambda x:x.strip())
merge_df['text']=merge_df['text'].apply(lambda x:re.sub(" +"," ",x))
exclude=set(string.punctuation)
merge_df['text']=merge_df['text'].apply(lambda x:''.join(ch for ch in x if ch not in exclude))
remove=str.maketrans('','',digits)
merge_df['text']=merge_df['text'].apply(lambda x:x.translate(remove))

In [ ]:
merge_df

,student_id,prompt_id,prompt_question,prompt_text,text,content,wording
0,000e8c3c7ddb,814d6b,summarize how the third wave developed over su...,Background \r\nThe Third Wave experiment took ...,the third wave was an experimentto see how peo...,0.205683,0.380538
1,0070c9e7af47,814d6b,summarize how the third wave developed over su...,Background \r\nThe Third Wave experiment took ...,the third wave developed rapidly because the s...,3.272894,3.219757
2,0095993991fe,814d6b,summarize how the third wave developed over su...,Background \r\nThe Third Wave experiment took ...,the third wave only started as an experiment w...,0.205683,0.380538
3,00c20c6ddd23,814d6b,summarize how the third wave developed over su...,Background \r\nThe Third Wave experiment took ...,the experimen was orginally about how even whe...,0.567975,0.969062
4,00d40ad10dc9,814d6b,summarize how the third wave developed over su...,Background \r\nThe Third Wave experiment took ...,the third wave developed so quickly due to the...,-0.910596,-0.081769
...,...,...,...,...,...,...,...
7160,fef3e85236e5,39c16e,summarize at least elements of an ideal trage...,Chapter 13 \r\nAs the sequel to what has alrea...,it has to be made on a complex storyline with ...,-0.981265,-1.548900
7161,ff0f65eecf02,39c16e,summarize at least elements of an ideal trage...,Chapter 13 \r\nAs the sequel to what has alrea...,aristotle descirbes an ideal tradgedy as being...,-0.511077,-1.589115
7162,ff186473ea0a,39c16e,summarize at least elements of an ideal trage...,Chapter 13 \r\nAs the sequel to what has alrea...,a tragedy should have a complex plan not a sim...,-0.834946,-0.593749
7163,ff5e9e6068da,39c16e,summarize at least elements of an ideal trage...,Chapter 13 \r\nAs the sequel to what has alrea...,aristotle believed that the ideal tradegy shou...,-0.157460,-0.165811


In [ ]:
prompt=[]
question=[]
for ch in merge_df['prompt_question']:
   tokens=nltk.word_tokenize(ch)
   prompt.append(tokens)
for x in prompt:
   #if x not in question:
      question.append(x)

In [ ]:
text=[]
answer=[]
for ch in merge_df['text']:
   tokens=nltk.word_tokenize(ch)
   text.append(tokens)
for x in text:
    #if x not in answer:
      answer.append(x)

In [ ]:
tokens=list(set([token for sublist in answer for token in sublist]))
answer_index={token:i for i,token in enumerate(tokens)}
Tokens=list(set([token for sublist in question for token in sublist]))
question_index={token:i for i,token in enumerate(Tokens)}


In [ ]:
answer_encoded_hot=np.zeros((len(answer),len(tokens)),dtype=np.float64)
question_encoded_hot=np.zeros((len(question),len(Tokens)),dtype=np.float64)


In [ ]:
for i,ch in enumerate(answer):
  for token in ch:
    answer_encoded_hot[i,answer_index[token]]=1
for i,ch in enumerate(question):
  for token in ch:
    question_encoded_hot[i,question_index[token]]=1



In [ ]:
array_of_answers = [row for row in answer_encoded_hot ]
array_of_question = [row for row in question_encoded_hot ]
max_shape = max(arr.shape for arr in array_of_answers)
arrays_answer = [np.pad(arr, [(0, max_shape[0] - arr.shape[0])], mode='constant') for arr in array_of_answers]
arrays_answer = [arr.astype(np.float64) for arr in arrays_answer]
max_shape = max(arr.shape for arr in array_of_question)
arrays_question = [np.pad(arr, [(0, max_shape[0] - arr.shape[0])], mode='constant') for arr in array_of_question]
arrays_question = [arr.astype(np.float64) for arr in arrays_question]


In [ ]:
merge_df['answer']=arrays_answer
merge_df['question']=arrays_question
merge_df=merge_df.loc[:,['student_id','prompt_id','question','answer','content','wording']]
merge_df

,student_id,prompt_id,question,answer,content,wording
0,000e8c3c7ddb,814d6b,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.205683,0.380538
1,0070c9e7af47,814d6b,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",3.272894,3.219757
2,0095993991fe,814d6b,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.205683,0.380538
3,00c20c6ddd23,814d6b,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0.567975,0.969062
4,00d40ad10dc9,814d6b,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.910596,-0.081769
...,...,...,...,...,...,...
7160,fef3e85236e5,39c16e,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.981265,-1.548900
7161,ff0f65eecf02,39c16e,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.511077,-1.589115
7162,ff186473ea0a,39c16e,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.834946,-0.593749
7163,ff5e9e6068da,39c16e,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",-0.157460,-0.165811


In [ ]:
X1=arrays_question
X2=arrays_answer
X=np.column_stack((X1,X2))
y=np.array(merge_df['wording']).ravel()
train_X, val_X, train_y, val_y = train_test_split(X,y,test_size = 0.25, random_state = 14)


In [ ]:
Model_wordings=RandomForestRegressor()
Model_wordings.fit(train_X,train_y)
predicted_wordings = Model_wordings.predict(val_X)

In [ ]:
predicted_wordings

array([-0.81923156, -0.76119953, -0.5650102 , ..., -0.63457543,
        0.38332407, -0.26359021])

In [ ]:
mae = mean_absolute_error(val_y,predicted_wordings)
mae

0.5699161648518756

In [ ]:
X1=arrays_question
X2=arrays_answer
X=np.column_stack((X1,X2))
y=np.array(merge_df['content']).ravel()
train_X, val_X, train_y, val_y = train_test_split(X,y,test_size = 0.25, random_state = 14)
Model_content=RandomForestRegressor()
Model_content.fit(train_X,train_y)
predicted_content = Model_content.predict(val_X)

In [ ]:
predicted_content

array([ 0.14086654, -0.44240293, -0.14202434, ..., -0.53767136,
        0.79273829,  0.19302356])

In [ ]:
mae = mean_absolute_error(val_y,predicted_content)
mae

0.4800999229641114

In [ ]:
merge_df2=pd.merge(df3,df4, on='prompt_id',how='inner')
merge_df2

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text
0,000000ffffff,abc123,Example text 1,Summarize...,Example Title 1,Heading\nText...
1,222222cccccc,abc123,Example text 3,Summarize...,Example Title 1,Heading\nText...
2,111111eeeeee,def789,Example text 2,Summarize...,Example Title 2,Heading\nText...
3,333333dddddd,def789,Example text 4,Summarize...,Example Title 2,Heading\nText...


In [ ]:
merge_df2['prompt_question']=merge_df2['prompt_question'].astype(str)
merge_df2['prompt_question']=merge_df2['prompt_question'].apply(lambda x:x.lower())
merge_df2['prompt_question']=merge_df2['prompt_question'].apply(lambda x:re.sub("'"," ", x))
merge_df2['prompt_question']=merge_df2['prompt_question'].apply(lambda x:x.strip())
merge_df2['prompt_question']=merge_df2['prompt_question'].apply(lambda x:re.sub(" +"," ",x))
exclude=set(string.punctuation)
merge_df2['prompt_question']=merge_df2['prompt_question'].apply(lambda x:''.join(ch for ch in x if ch not in exclude))


In [ ]:
merge_df2['text']=merge_df2['text'].astype(str)
merge_df2['text']=merge_df2['text'].apply(lambda x:x.lower())
merge_df2['text']=merge_df2['text'].apply(lambda x:re.sub("'"," ", x))
merge_df2['text']=merge_df2['text'].apply(lambda x:x.strip())
merge_df2['text']=merge_df2['text'].apply(lambda x:re.sub(" +"," ",x))
exclude=set(string.punctuation)
merge_df2['text']=merge_df2['text'].apply(lambda x:''.join(ch for ch in x if ch not in exclude))


In [ ]:

merge_df2

,student_id,prompt_id,text,prompt_question,prompt_title,prompt_text
0,000000ffffff,abc123,example text 1,summarize,Example Title 1,Heading\nText...
1,222222cccccc,abc123,example text 3,summarize,Example Title 1,Heading\nText...
2,111111eeeeee,def789,example text 2,summarize,Example Title 2,Heading\nText...
3,333333dddddd,def789,example text 4,summarize,Example Title 2,Heading\nText...


In [ ]:
prompt=[]
question_test=[]
for ch in merge_df2['prompt_question']:
   tokens=nltk.word_tokenize(ch)
   prompt.append(tokens)
for x in prompt:
   #if x not in question:
      question_test.append(x)

In [ ]:
prompt=[]
answer_test=[]
for ch in merge_df2['text']:
   tokens=nltk.word_tokenize(ch)
   prompt.append(tokens)
for x in prompt:
   #if x not in question:
      answer_test.append(x)

In [ ]:
tokens=list(set([token for sublist in answer_test for token in sublist]))
answer_test_index={token:i for i,token in enumerate(tokens)}
Tokens=list(set([token for sublist in question_test for token in sublist]))
question_test_index={token:i for i,token in enumerate(Tokens)}


In [ ]:
answer_encoded_test=np.zeros((len(answer_test),len(tokens)),dtype=np.float64)
question_encoded_test=np.zeros((len(question_test),len(Tokens)),dtype=np.float64)


In [ ]:
for i,ch in enumerate(answer_test):
  for token in ch:
    answer_encoded_test[i,answer_test_index[token]]=1
for i,ch in enumerate(question_test):
  for token in ch:
    question_encoded_test[i,question_test_index[token]]=1


In [ ]:
array_answers_test = [row for row in answer_encoded_test ]
array_questions_test = [row for row in question_encoded_test ]
max_shape = max(arr.shape for arr in array_answers_test)
arrays_answer_test = [np.pad(arr, [(0, max_shape[0] - arr.shape[0])], mode='constant') for arr in array_answers_test]
arrays_answer_test = [arr.astype(np.float64) for arr in arrays_answer_test]
max_shape = max(arr.shape for arr in array_questions_test)
arrays_question_test = [np.pad(arr, [(0, max_shape[0] - arr.shape[0])], mode='constant') for arr in array_questions_test]
arrays_question_test = [arr.astype(np.float64) for arr in arrays_question_test]

In [ ]:
Student_id=merge_df2['student_id'].values
merge_df2['answer']=arrays_answer_test
merge_df2['question']=arrays_question_test
merge_df2=merge_df2.loc[:,['student_id','prompt_id','question','answer']]
merge_df2

,student_id,prompt_id,question,answer
0,000000ffffff,abc123,[1.0],"[0.0, 1.0, 1.0, 0.0, 0.0, 1.0]"
1,222222cccccc,abc123,[1.0],"[0.0, 1.0, 1.0, 0.0, 1.0, 0.0]"
2,111111eeeeee,def789,[1.0],"[1.0, 1.0, 1.0, 0.0, 0.0, 0.0]"
3,333333dddddd,def789,[1.0],"[0.0, 1.0, 1.0, 1.0, 0.0, 0.0]"


In [ ]:
X1_test=arrays_question_test
X2_test=arrays_answer_test
x_test=np.column_stack((X1_test,X2_test))

In [ ]:
columns_to_pad=train_X.shape[1]-x_test.shape[1]
x_test_padded=np.pad(x_test,((0,0),(0,columns_to_pad)),mode='constant',constant_values=0)


In [ ]:
y_pred_test_wordings=Model_wordings.predict(x_test_padded)

In [ ]:
y_pred_test_wordings

array([-1.4368077, -1.4368077, -1.4368077, -1.4368077])

In [ ]:
y_pred_test_content=Model_content.predict(x_test_padded)

In [ ]:
y_pred_test_content

array([-1.43341332, -1.43325321, -1.43437579, -1.43437579])

In [ ]:
Content=y_pred_test_content
Wording=y_pred_test_wordings
file_n='Sample_Submission.csv'
with open(file_n,'w',newline='') as csvfile:
  csv_w=csv.writer(csvfile)
  csv_w.writerow(['Student_id','Content','Wording'])
  for Student_id,Content,Wording in zip(Student_id,Content,Wording):
    csv_w.writerow([Student_id,Content,Wording])